In [1]:
!pip install bert-score --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.1 MB/s eta 0:00:00


In [2]:
# Importing paths
import pandas as pd
from bert_score import score as bert_score
import torch

In [3]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
!wget https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
!tar -xf abo-images-small.tar

--2025-05-18 15:56:37--  https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
Resolving amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)... 52.217.139.9, 16.182.65.217, 3.5.16.57, ...
Connecting to amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)|52.217.139.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3253381120 (3.0G) [application/x-tar]
Saving to: ‘abo-images-small.tar’

abo-images-small.ta 100%[===================>]   3.03G  17.7MB/s    in 3m 0s   

2025-05-18 15:59:38 (17.2 MB/s) - ‘abo-images-small.tar’ saved [3253381120/3253381120]



### **Evaluating for QWEN7B**

In [5]:
# Set paths
CSV_PATH = "/content/drive/MyDrive/images/Results/results_QWEN7B.csv"
IMAGE_ROOT = "/content/images/small/"

# Load predictions CSV
df = pd.read_csv(CSV_PATH)

In [6]:
# Lowercase both predictions and ground truth for string matching
df['answer'] = df['answer'].astype(str).str.lower().str.strip()
df['generated_answer'] = df['generated_answer'].astype(str).str.lower().str.strip()

In [7]:
df.head(10)

,image_path,question,answer,generated_answer
0,a1/a1f37cc0.jpg,What is the pattern?,comforter,solid
1,77/772f6f1f.jpg,What brand is this?,206,ugg
2,1d/1d3c1567.jpg,What is the brand?,solimo,mtt
3,fb/fb57539f.jpg,Is it 3D?,yes,no
4,e0/e030ba61.jpg,What is the product type?,grocery,100% arabica coffee beans
5,49/49b1b22b.jpg,What type of case is it?,hard,optimal lift
6,1f/1f243ec0.jpg,What kind of shoe is it?,running,leone
7,50/50ce84a2.jpg,What type of wash?,handwash,liquid handwash
8,49/49b1b22b.jpg,What is the product's brand?,solimo,apple
9,0f/0fc9d6c7.jpg,What is the product type?,case,phone case


In [8]:
# String Matching Accuracy
df['is_match'] = df['answer'] == df['generated_answer']
string_match_accuracy = df['is_match'].mean()

In [9]:
# BERTScore
references = df['answer'].tolist()
candidates = df['generated_answer'].tolist()

P, R, F1 = bert_score(candidates, references, lang='en', device='cuda' if torch.cuda.is_available() else 'cpu')
average_bertscore = F1.mean().item()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from nltk.corpus import wordnet as wn

def wup_similarity(word1, word2):
    """
    Compute Wu-Palmer similarity between the first synset of word1 and word2.
    Returns 0 if no synsets are found.
    """
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)
    if synsets1 and synsets2:
        max_sim = max((s1.wup_similarity(s2) or 0) for s1 in synsets1 for s2 in synsets2)
        return max_sim
    return 0

# Prepare lists
references = df['answer'].tolist()
candidates = df['generated_answer'].tolist()

# Calculate WUP similarities
wup_scores = [wup_similarity(ref, pred) for ref, pred in zip(references, candidates)]
average_wup = sum(wup_scores) / len(wup_scores)

In [11]:
# Display Results
print(f"String Matching Accuracy: {string_match_accuracy:.4f}")
print(f"Average BERTScore (F1):   {average_bertscore:.4f}")
print(f" Average WUP Similarity: {average_wup:.4f}")

String Matching Accuracy: 0.2710
Average BERTScore (F1):   0.9192
 Average WUP Similarity: 0.4177


### **Evaluating for QWEN2B**

In [12]:
# Set paths
CSV_PATH = "/content/drive/MyDrive/images/Results/results_qwen2B.csv"

In [13]:
# Load predictions CSV
df = pd.read_csv(CSV_PATH)

In [14]:
# Lowercase both predictions and ground truth for string matching
df['answer'] = df['answer'].astype(str).str.lower().str.strip()
df['generated_answer'] = df['generated_answer'].astype(str).str.lower().str.strip()

In [15]:
df.head(5)

,image_path,question,answer,generated_answer
0,a1/a1f37cc0.jpg,What is the pattern?,comforter,dance
1,77/772f6f1f.jpg,What brand is this?,206,apple
2,1d/1d3c1567.jpg,What is the brand?,solimo,tangram
3,fb/fb57539f.jpg,Is it 3D?,yes,no
4,e0/e030ba61.jpg,What is the product type?,grocery,clothing


In [16]:
# String Matching Accuracy
df['is_match'] = df['answer'] == df['generated_answer']
string_match_accuracy = df['is_match'].mean()

In [17]:
# BERTScore
references = df['answer'].tolist()
candidates = df['generated_answer'].tolist()

P, R, F1 = bert_score(candidates, references, lang='en', device='cuda' if torch.cuda.is_available() else 'cpu')
average_bertscore = F1.mean().item()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from nltk.corpus import wordnet as wn

def wup_similarity(word1, word2):
    """
    Compute Wu-Palmer similarity between the first synset of word1 and word2.
    Returns 0 if no synsets are found.
    """
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)
    if synsets1 and synsets2:
        max_sim = max((s1.wup_similarity(s2) or 0) for s1 in synsets1 for s2 in synsets2)
        return max_sim
    return 0

# Prepare lists
references = df['answer'].tolist()
candidates = df['generated_answer'].tolist()

# Calculate WUP similarities
wup_scores = [wup_similarity(ref, pred) for ref, pred in zip(references, candidates)]
average_wup = sum(wup_scores) / len(wup_scores)

In [19]:
# Display Results
print(f"String Matching Accuracy: {string_match_accuracy:.4f}")
print(f"Average BERTScore (F1):   {average_bertscore:.4f}")
print(f" Average WUP Similarity: {average_wup:.4f}")

String Matching Accuracy: 0.1032
Average BERTScore (F1):   0.9288
 Average WUP Similarity: 0.4123
